In [1]:
import gym
env = gym.make('CartPole-v1')
# The observation space is `Box(4,)`, a 4-element vector.
observation_size = env.observation_space.shape[0]

In [26]:
import tensorflow.compat.v1 as tf1
import os
tf1.disable_v2_behavior()
#import tensorflow as tf #Tensorflow 2.x


#resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
#tf.contrib.distribute.initialize_tpu_system(resolver)
#strategy = tf.contrib.distribute.TPUStrategy(resolver) 

def dense_nn(inputs, layers_sizes, scope_name):
    """Creates a densely connected multi-layer neural network.
    inputs: the input tensor
    layers_sizes (list<int>): defines the number of units in each layer. The output 
        layer has the size layers_sizes[-1].
    """
    with tf1.variable_scope(scope_name):
        for i, size in enumerate(layers_sizes):
            inputs = tf1.layers.dense(
                inputs,
                size,
                # Add relu activation only for internal layers.
                # Rectified Linear Unit:  output the input directly if it is positive, otherwise
                activation='relu' if i < len(layers_sizes) - 1 else None,
                #kernel_initializer=tf1.contrib.layers.xavier_initializer(),
                kernel_initializer=tf1.initializers.random_uniform(),
                name=scope_name + '_l' + str(i)
            )
    return inputs

#with tf1.variable_scope('scope_name'):
for i, size in enumerate([1,2]):
            inputs = tf1.layers.dense(
                inputs = states,
                units = size,
                # Add relu activation only for internal layers.
                # Rectified Linear Unit:  output the input directly if it is positive, otherwise
                activation='relu' if i < len([1,2]) - 1 else None,
                #kernel_initializer=tf1.contrib.layers.xavier_initializer(),
                kernel_initializer=tf1.initializers.random_uniform(),
                name='scope_name' + '_l' + str(i)
            )       
            
q = dense_nn(states, [32, 32, 2], scope_name='Q_primary')

ValueError: Variable scope_name_l1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-19-2a3016038bc4>", line 1, in <module>
    a = tf1.layers.dense(
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3071, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)


In [19]:
a = tf1.layers.dense(
                states,
                32,
                # Add relu activation only for internal layers.
                # Rectified Linear Unit:  output the input directly if it is positive, otherwise
                activation='relu' if True else None,
                #kernel_initializer=tf1.contrib.layers.xavier_initializer(),
                kernel_initializer=tf1.initializers.random_uniform(),
                name='scope_name' + '_l' + str(1)
            )
a


<tf.Tensor 'scope_name_l1/Relu:0' shape=(32, 32) dtype=float32>

In [6]:
batch_size = 32  # A tunable hyperparameter. # no. input obs
#A placeholder is simply a variable that we will assign data to at a later date. 
#It allows us to create our operations and build our computation graph, without needing the data.
#Inserts a placeholder for a tensor that will be always fed.
# tensor: multi dimesional tensor

states = tf1.placeholder(tf1.float32, shape=(batch_size, observation_size), name='state')
states_next = tf1.placeholder(tf1.float32, shape=(batch_size, observation_size), name='state_next')
actions = tf1.placeholder(tf1.int32, shape=(batch_size,), name='action')
rewards = tf1.placeholder(tf1.float32, shape=(batch_size,), name='reward')
done_flags = tf1.placeholder(tf1.float32, shape=(batch_size,), name='done')


#states.shape

In [17]:
q = dense_nn(states, [32, 32, 2], scope_name='Q_primary')
q_target = dense_nn(states_next, [32, 32, 2], scope_name='Q_target')
q

ValueError: Variable Q_primary/Q_primary_l0/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-15-a0d7c9f029f6>", line 19, in dense_nn
    inputs = tf1.layers.dense(
  File "<ipython-input-16-8fe27e16d5e2>", line 1, in <module>
    q = dense_nn(states, [32, 32, 2], scope_name='Q_primary')
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3263, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3071, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
# The prediction by the primary Q network for the actual actions.
action_one_hot = tf.one_hot(actions, 1.0, 1.0, 0.0, name='action_one_hot')
pred = tf.reduce_sum(q * action_one_hot, reduction_indices=-1, name='q_acted')

# The optimization target defined by the Bellman equation and the target network.
max_q_next_by_target = tf.reduce_max(q_target, axis=-1)
y = rewards + (1. - done_flags) * gamma * max_q_next_by_target

# The loss measures the mean squared error between prediction and target.
loss = tf.reduce_mean(tf.square(pred - tf.stop_gradient(y)), name="loss_mse_train")
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss, name="adam_optim")

TypeError: Expected int32 passed to parameter 'depth' of op 'OneHot', got 1.0 of type 'float' instead. Error: Expected int32, got 1.0 of type 'float' instead.